In [1]:
import pandas as pd
import sys
import logging
import json
from llm_api_client import LLMAPIClient

In [2]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

logger = logging.getLogger(__name__)

In [3]:
prompt = """다음은 학생이 제출한 수행평가 과제물이다. 제시된 평가 기준을 참고하여 과제물을 체계적으로 분석하고 평가하라. 평가 시 다음 단계를 순차적으로 따르라:

1. 평가기준을 꼼꼼히 읽고 이해하라.
2. 학생의 제출물을 세심히 분석하며, 평가 기준에 부합하거나 미흡한 점이 있는지 확인하라. 만약 직접적으로 인용할 수 있는 표현이 있다면, 해당 표현을 정확히 인용하라.
3. 인용한 표현이 있다면 평가 기준과의 관계를 논리적으로 설명하라.
4. 평가 항목별로 정량적 점수를 부여하며 근거를 구체적으로 기술하라.

학생 제출물:
[학생 제출물]"""

system_instruction ="""너는 고등학교 [과목] 담당 교사이며, 학생이 제출한 [수행평가 제목] 수행평가 과제물을 객관적이고 공정하게 평가하는 전문 평가자 역할을 수행한다. 다음 지침을 명확히 숙지하여 평가하라.
수행평가에 대한 세부정보는 다음과 같다:

학년: [학년]
수행평가 제목: [수행평가 제목]
수행평가 유형: [수행평가 유형]
수행평가 설명: [수행평가 설명]
평가 기준:
[평가 기준]

평가 수행 지침
1. 평가 기준 엄격 준수
- 제공된 평가 기준 및 체크리스트의 항목을 정확히 준수하여 평가하라.

2. 직접적 인용 및 명확한 설명
- 학생의 제출물에서 인용된 표현을 근거로 사용한다면 해당 표현이 평가 기준에 적합하거나 미흡한 이유를 구체적으로 설명하라.

3. 논리적이고 객관적인 근거
- 평가 결과가 일관되도록 하고, 점수 부여의 근거를 명확히 기술하여 객관성과 신뢰성을 높이도록 하라.

결과는 JSON 객체로만 출력하라. 마크다운 코드 펜스(```)는 포함하지 말고, 모든 문자열이 올바르게 따옴표로 감싸지도록 하라."""

output_schema = {
    "type": "object",
    "properties": {
        "checklist": {
            "type": "array",
            "description": "평가 항목별 점수와 이유",
            "items": {
                "type": "object",
                "properties": {
                    "title": {
                        "type": "string",
                        "description": "평가 대분류"
                    },
                    "subcategories": {
                        "type": "array",
                        "items": {
                            "type": "object",
                            "properties": {
                                "name": {
                                    "type": "string",
                                    "description": "평가 소분류"
                                },
                                "score": {
                                    "type": "integer",
                                    "description": "0~3 사이의 점수"
                                },
                                "reason": {
                                    "type": "string",
                                    "description": "점수 평가 이유"
                                },
                                "evidence":{
                                    "type": "array",
                                    "description": "관련 있는 텍스트를 증거로 제시",
                                    "items": {
                                        "type": "string",
                                        "description": "증거 텍스트"
                                    }
                                }
                            },
                            "required": ["name", "score", "reason", "evidence"]
                        }
                    }
                },
                "required": ["title", "subcategories"]
            }
        },
        "overall_feedback": {
            "type": "string",
            "description": "전체적인 피드백"
        }
    },
    "required": ["checklist", "overall_feedback"]
}

In [4]:
df_aihub = pd.read_csv('C:/Users/yuhan/Desktop/CSD_18기/베어러블/AI_Hub/에세이_글_평가_데이터_전처리완.csv')

good_essay_ids = ["ESSAY_71375", "ESSAY_65900", "ESSAY_43915"]
bad_essay_ids = ["ESSAY_78467", "ESSAY_76539", "ESSAY_76550"]
# essay_dict = df_aihub.iloc[3000].to_dict()
essay_dict = {}
for essay_id in good_essay_ids + bad_essay_ids:
    essay_dict[essay_id] = df_aihub.loc[df_aihub['essay_id'] == essay_id, :].to_dict(orient='records')[0]

In [ ]:
with open('checklists/논술형_checklist.json', 'r', encoding='utf-8') as f:
    checklist = json.load(f)

def format_template(template, values, checklist):
    """Format a template with the given values
    
    Args:
        template: The template string with [placeholders]
        values: Dictionary of placeholder values
        
    Returns:
        Formatted template with placeholders replaced
    """
    formatted = template
    replacements = {
        '학년': values.get('student_grade', ''),
        '과목': values.get('subject', '국어'),
        '수행평가 제목': values.get('essay_main_subject', ''),
        '수행평가 유형': values.get('essay_type', ''),
        '수행평가 설명': values.get('essay_prompt', ''),
        '평가 기준': values.get('checklist', json.dumps(checklist, ensure_ascii=False, indent=2)),
        '학생 제출물': values.get('processed_text', '')
    }
    
    for placeholder, value in replacements.items():
        formatted = formatted.replace(f"[{placeholder}]", value)
        
    return formatted

In [6]:
# Initialize the client
client = LLMAPIClient()
client.logger = logger

2025-06-21 15:08:13,211 - INFO - Logging initialized. Log file: logs\api_interactions_20250621_150813.log
2025-06-21 15:08:13,916 - INFO - LLM API Client initialized


In [7]:
models_list = [
    "gemini-2.5-flash",
    "gemini-2.5-flash-lite-preview-06-17",
    "gemini-2.0-flash",
    "gemini-2.0-flash-lite",
    "claude-sonnet-4-20250514",
    "claude-3-7-sonnet-20250219",
    "claude-3-5-haiku-20241022",
    "claude-3-5-sonnet-20241022",
    "claude-3-haiku-20240307",
    "gpt-4.1",
    "gpt-4.1-mini",
    "gpt-4.1-nano",
    "gpt-4o",
    "gpt-4o-mini",
    "o4-mini",
    "o3-mini"
]

In [ ]:
import time

all_scores = []

for essay_id, essay_data in essay_dict.items():
    for trial in range(1, 6):
        formatted_prompt = format_template(prompt, essay_data, checklist)
        formatted_system_instruction = format_template(system_instruction, essay_data, checklist)

        params = {
            "prompt": formatted_prompt,
            "system_instruction": formatted_system_instruction,
            "models": models_list,
            "response_schema": output_schema,
            "file_path": None,
            "temperature": 0.1,
            "max_tokens": 4096,
            "enable_thinking": False
        }

        essay_results = await client.generate_responses(**params)

        client.save_results(essay_results, file_path=f"{essay_id}_evaluation_results_trial_{trial}.json")

        processed_results = client.process_results(essay_results)
        with open(f"{essay_id}_processed_results_trial_{trial}.json", 'w', encoding='utf-8') as f:
            json.dump(processed_results, f, ensure_ascii=False, indent=2)

        essay_scores_df = client.extract_scores(essay_results, essay_id=essay_id)
        essay_scores_df.insert(loc=1, column='trial', value=trial)
        all_scores.append(essay_scores_df)

        time.sleep(30)

# scores_df = pd.concat(all_scores, ignore_index=True)

2025-06-21 15:08:13,964 - INFO - Processing with Gemini model: gemini-2.5-flash
2025-06-21 15:08:13,967 - INFO - Gemini 모델에 요청 전송 중: gemini-2.5-flash
2025-06-21 15:08:13,967 - INFO - Processing with Gemini model: gemini-2.5-flash-lite-preview-06-17
2025-06-21 15:08:13,968 - INFO - AFC is enabled with max remote calls: 10.
2025-06-21 15:08:13,968 - INFO - Processing with Gemini model: gemini-2.0-flash
2025-06-21 15:08:13,970 - INFO - Processing with Gemini model: gemini-2.0-flash-lite
2025-06-21 15:08:13,972 - INFO - Processing with Anthropic model: claude-sonnet-4-20250514
2025-06-21 15:08:13,972 - INFO - Gemini 모델에 요청 전송 중: gemini-2.5-flash-lite-preview-06-17
2025-06-21 15:08:13,972 - INFO - Processing with Anthropic model: claude-3-7-sonnet-20250219
2025-06-21 15:08:13,973 - INFO - Processing with Anthropic model: claude-3-5-haiku-20241022
2025-06-21 15:08:13,977 - INFO - Processing with Anthropic model: claude-3-5-sonnet-20241022
2025-06-21 15:08:13,979 - INFO - Processing with Anth

In [53]:
# Read log file logs\api_interactions_20250621_150813.log
# Only read in ERROR and WARNING logs
error_logs = []
with open('logs/api_interactions_20250621_150813.log', 'r', encoding='utf-8') as f:
    for line in f:
        if 'ERROR' in line or 'WARNING' in line or 'Results saved to' in line:
            error_logs.append(line.strip())

| Essay ID | Model                              | Trial | Error Type                            |
|----------|-------------------------------------|-------|----------------------------------------|
| 71375    | o3-mini                             | 5     | Incomplete response (token limit)      |
| 65900    | gemini-2.5-flash-lite-preview-06-17 | 1     | Incomplete response (token limit)      |
| 65900    | gemini-2.5-flash-lite-preview-06-17 | 2     | NoneType object (subscript)            |
| 65900    | gpt-4.1-mini                        | 2     | Incomplete response (token limit)      |
| 65900    | gemini-2.5-flash-lite-preview-06-17 | 3     | Incomplete response (token limit)      |
| 65900    | gemini-2.5-flash-lite-preview-06-17 | 4     | Incomplete response (token limit)      |
| 65900    | gemini-2.5-flash-lite-preview-06-17 | 5     | NoneType object (subscript)            |
| 43915    | gpt-4.1-nano                        | 1     | Incomplete response (token limit)      |
| 43915    | claude-3-5-sonnet-20241022          | 5     | Incomplete response (token limit)      |
| 78467    | claude-sonnet-4-20250514            | 1     | Incomplete response (token limit)      |
| 78467    | claude-3-haiku-20240307             | 1     | Incomplete response (token limit)      |
| 78467    | gemini-2.5-flash                    | 2     | Incomplete response (token limit)      |
| 78467    | gemini-2.5-flash-lite-preview-06-17 | 2     | Incomplete response (token limit)      |
| 78467    | claude-sonnet-4-20250514            | 2     | Incomplete response (token limit)      |
| 78467    | claude-3-haiku-20240307             | 2     | Incomplete response (token limit)      |
| 78467    | claude-sonnet-4-20250514            | 3     | Incomplete response (token limit)      |
| 78467    | claude-3-haiku-20240307             | 3     | Incomplete response (token limit)      |
| 78467    | claude-sonnet-4-20250514            | 4     | Incomplete response (token limit)      |
| 78467    | claude-3-haiku-20240307             | 4     | Incomplete response (token limit)      |
| 78467    | gemini-2.5-flash                    | 5     | Incomplete response (token limit)      |
| 78467    | claude-sonnet-4-20250514            | 5     | Incomplete response (token limit)      |
| 76539    | claude-3-haiku-20240307             | 1     | Incomplete response (token limit)      |
| 76539    | claude-3-haiku-20240307             | 2     | Incomplete response (token limit)      |
| 76539    | claude-3-haiku-20240307             | 3     | Incomplete response (token limit)      |
| 76539    | claude-3-haiku-20240307             | 4     | Incomplete response (token limit)      |
| 76539    | claude-3-haiku-20240307             | 5     | Incomplete response (token limit)      |
| 76550    | o3-mini                             | 1     | Incomplete response (token limit)      |

In [61]:
# Read the evaluation results from a specific file
with open('auto_evaluation_results\evaluation_results_json\ESSAY_43915_evaluation_results_trial_3.json', 'r', encoding='utf-8') as f:
    # Read lines after "Model: gpt-4.1-nano" and before "Model: gpt-4o"
    for i, line in enumerate(f):
        if 'Model: gpt-4.1-nano' in line:
            start_index = i + 1
        elif 'Model: gpt-4o' in line:
            end_index = i
            break
    evaluation_results = f.readlines()[start_index:end_index]

    for line in f:
        print(line.strip())

In [9]:
scores_df

,essay_id,trial,model,response_time,문법의 정확성,단어 사용의 적절성,문장 표현의 적절성,문단 내 구조의 적절성,문단 간 구조의 적절성,구조의 일관성,분량의 적절성,주제의 명료성,설명의 구체성,사고의 창의성,프롬프트 독해,구성,내용
0,ESSAY_71375,1,gemini-2.5-flash,10.044278,2,2,2,2.0,1.0,2.0,3.0,2.0,2.0,3.0,3.0,NaN,NaN
1,ESSAY_71375,1,gemini-2.5-flash-lite-preview-06-17,5.595316,2,2,2,2.0,2.0,2.0,2.0,2.0,1.0,2.0,3.0,NaN,NaN
2,ESSAY_71375,1,gemini-2.0-flash,9.249181,2,2,2,2.0,2.0,2.0,3.0,3.0,2.0,2.0,3.0,NaN,NaN
3,ESSAY_71375,1,gemini-2.0-flash-lite,9.981756,2,2,2,2.0,1.0,1.0,2.0,2.0,2.0,2.0,3.0,NaN,NaN
4,ESSAY_71375,1,claude-sonnet-4-20250514,39.865103,2,2,1,1.0,0.0,1.0,2.0,1.0,2.0,2.0,2.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
448,ESSAY_76550,5,gpt-4.1-nano,7.318925,2,2,2,2.0,2.0,2.0,2.0,3.0,2.0,2.0,3.0,NaN,NaN
449,ESSAY_76550,5,gpt-4o,5.831265,2,2,2,2.0,2.0,2.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN
450,ESSAY_76550,5,gpt-4o-mini,14.312863,2,2,2,2.0,2.0,2.0,3.0,3.0,2.0,2.0,3.0,NaN,NaN
451,ESSAY_76550,5,o4-mini,31.211178,2,2,2,2.0,2.0,2.0,3.0,3.0,2.0,2.0,3.0,NaN,NaN


In [10]:
scores_df.to_csv('essay_evaluation_scores.csv', index=False, encoding='utf-8-sig')

In [42]:
models_to_remove = set()

In [43]:
# Count how many times each model was used
model_counts = scores_df['model'].value_counts().reset_index()
model_counts.columns = ['model', 'count']

# Identify models that were used 25 times or less
models_to_remove.update(model_counts[model_counts['count'] <= 25]['model'].tolist())

In [28]:
scores_std_df = scores_df.groupby(['essay_id', 'model'])[['문법의 정확성', '단어 사용의 적절성','문장 표현의 적절성', '문단 내 구조의 적절성', '문단 간 구조의 적절성', '구조의 일관성', '분량의 적절성',
    '주제의 명료성', '설명의 구체성', '사고의 창의성', '프롬프트 독해']].std()

scores_avg_std_df = scores_std_df.groupby('model')[['문법의 정확성', '단어 사용의 적절성','문장 표현의 적절성', '문단 내 구조의 적절성', '문단 간 구조의 적절성', '구조의 일관성', '분량의 적절성',
    '주제의 명료성', '설명의 구체성', '사고의 창의성', '프롬프트 독해']].mean()
scores_avg_std_df['mean_std'] = scores_avg_std_df.mean(axis=1, numeric_only=True)

In [44]:
models_to_remove.update(scores_avg_std_df[scores_avg_std_df['mean_std'] >= 0.1].index.tolist())

In [45]:
models_to_remove

{'claude-3-7-sonnet-20250219',
 'claude-3-haiku-20240307',
 'claude-sonnet-4-20250514',
 'gemini-2.0-flash-lite',
 'gemini-2.5-flash',
 'gemini-2.5-flash-lite-preview-06-17',
 'gpt-4.1-mini',
 'o3-mini',
 'o4-mini'}

In [ ]:
set(models_list) - models_to_remove

{'claude-3-5-haiku-20241022',
 'claude-3-5-sonnet-20241022',
 'gemini-2.0-flash',
 'gpt-4.1',
 'gpt-4.1-nano',
 'gpt-4o',
 'gpt-4o-mini'}

In [ ]:
gpt_4_1_scores = scores_df[(scores_df['model'] == 'gpt-4.1')].sort_values(by=['essay_id', 'trial']).reset_index(drop=True)
gpt_4_1_scores

,essay_id,trial,model,response_time,문법의 정확성,단어 사용의 적절성,문장 표현의 적절성,문단 내 구조의 적절성,문단 간 구조의 적절성,구조의 일관성,분량의 적절성,주제의 명료성,설명의 구체성,사고의 창의성,프롬프트 독해,구성,내용
0,ESSAY_43915,1,gpt-4.1,11.297020,2,2,2,2.0,2.0,2.0,3.0,3.0,2.0,1.0,3.0,NaN,NaN
1,ESSAY_43915,2,gpt-4.1,12.436117,2,2,2,2.0,2.0,2.0,3.0,3.0,3.0,1.0,3.0,NaN,NaN
2,ESSAY_43915,3,gpt-4.1,11.833509,3,2,2,2.0,2.0,2.0,3.0,3.0,2.0,1.0,3.0,NaN,NaN
3,ESSAY_43915,4,gpt-4.1,13.147908,2,2,2,2.0,2.0,2.0,3.0,3.0,2.0,1.0,3.0,NaN,NaN
4,ESSAY_43915,5,gpt-4.1,9.859612,3,2,2,2.0,2.0,2.0,3.0,3.0,2.0,1.0,3.0,NaN,NaN
5,ESSAY_65900,1,gpt-4.1,11.021901,2,2,2,2.0,2.0,2.0,3.0,3.0,2.0,2.0,3.0,NaN,NaN
6,ESSAY_65900,2,gpt-4.1,13.769382,2,2,2,2.0,2.0,2.0,3.0,3.0,2.0,2.0,3.0,NaN,NaN
7,ESSAY_65900,3,gpt-4.1,11.240769,2,2,2,2.0,2.0,2.0,3.0,3.0,2.0,2.0,3.0,NaN,NaN
8,ESSAY_65900,4,gpt-4.1,10.791006,2,2,2,2.0,2.0,2.0,3.0,3.0,2.0,2.0,3.0,NaN,NaN
9,ESSAY_65900,5,gpt-4.1,11.580285,2,2,2,2.0,2.0,2.0,3.0,3.0,2.0,2.0,3.0,NaN,NaN


In [35]:
gemini_2_5_flash_lite_scores = scores_df[(scores_df['model'] == 'gemini-2.5-flash-lite-preview-06-17')].sort_values(by=['essay_id', 'trial']).reset_index(drop=True)
gemini_2_5_flash_lite_scores

,essay_id,trial,model,response_time,문법의 정확성,단어 사용의 적절성,문장 표현의 적절성,문단 내 구조의 적절성,문단 간 구조의 적절성,구조의 일관성,분량의 적절성,주제의 명료성,설명의 구체성,사고의 창의성,프롬프트 독해,구성,내용
0,ESSAY_43915,1,gemini-2.5-flash-lite-preview-06-17,4.338902,2,2,2,2.0,2.0,2.0,3.0,3.0,2.0,1.0,3.0,NaN,NaN
1,ESSAY_43915,2,gemini-2.5-flash-lite-preview-06-17,4.792323,2,2,2,2.0,3.0,2.0,3.0,3.0,2.0,1.0,3.0,NaN,NaN
2,ESSAY_43915,3,gemini-2.5-flash-lite-preview-06-17,5.477309,2,2,2,2.0,2.0,2.0,2.0,3.0,2.0,1.0,3.0,NaN,NaN
3,ESSAY_43915,4,gemini-2.5-flash-lite-preview-06-17,5.854286,2,2,2,2.0,2.0,2.0,2.0,3.0,2.0,1.0,3.0,NaN,NaN
4,ESSAY_43915,5,gemini-2.5-flash-lite-preview-06-17,5.509646,2,2,2,2.0,3.0,2.0,3.0,3.0,2.0,1.0,3.0,NaN,NaN
5,ESSAY_71375,1,gemini-2.5-flash-lite-preview-06-17,5.595316,2,2,2,2.0,2.0,2.0,2.0,2.0,1.0,2.0,3.0,NaN,NaN
6,ESSAY_71375,2,gemini-2.5-flash-lite-preview-06-17,4.787500,2,2,2,2.0,2.0,2.0,3.0,3.0,2.0,2.0,3.0,NaN,NaN
7,ESSAY_71375,3,gemini-2.5-flash-lite-preview-06-17,5.323538,2,2,2,2.0,2.0,2.0,3.0,3.0,2.0,2.0,3.0,NaN,NaN
8,ESSAY_71375,4,gemini-2.5-flash-lite-preview-06-17,5.275533,2,2,2,2.0,3.0,2.0,3.0,3.0,3.0,2.0,3.0,NaN,NaN
9,ESSAY_71375,5,gemini-2.5-flash-lite-preview-06-17,5.262284,2,2,2,2.0,2.0,2.0,2.0,3.0,2.0,2.0,3.0,NaN,NaN
